# ICE-NMF Hyperparameter tuning palco_2010

In [7]:
#imports
import numpy as np 
import csv
import re
import sys
import math
import matplotlib.pyplot as plt
from numpy import linalg as L
import pandas as pd
import random
from sklearn.preprocessing import normalize
from numba import njit
from collections import defaultdict
import random
from implicit_data import ImplicitData
import pandas as pd
from BISGD import BISGD
from ICE_NMF import ICE_NMF
from collections import defaultdict
from EvalPrequential import EvalPrequential
from datetime import datetime
import getopt

## Functions


In [8]:
def sigmoid(x):
  return 1 / (1 + math.exp(-x))

def sumvector(V,x):
    result = []
    for v in V:
        temp = v + x
        result.append(temp)
    return result    

def multvector(x,V):
    result = []
    for v in V:
        temp = v * x
        result.append(temp)
    return result    

def TransformVec(V):
  
    #new = normalize(V[:,np.newaxis], axis=0).ravel()
    for i in range(len(V)):
        V[i] = sigmoid(V[i])
    D = LA.norm(V)
    D=1/D
    new = D * V
  

    return new


## Get the data

In [9]:
sys.path.append('./')
configPath = 'config'
dataPath = 'palco_2010.tsv'


Data = []
data = pd.read_csv("palco_2010.tsv","\t")

Data = data.values.tolist()

print(data)



        user_id  track_id            timestamp
0          4220     31313  2010-01-01 00:10:36
1          4220     29567  2010-01-01 00:13:07
2          4220     31313  2010-01-01 00:13:18
3          3970     34925  2010-01-01 00:39:21
4          6258     18106  2010-01-01 00:46:25
...         ...       ...                  ...
588846    51303     37059  2010-12-31 23:43:41
588847    56256     66353  2010-12-31 23:44:15
588848    51303     12801  2010-12-31 23:46:58
588849    51303     12800  2010-12-31 23:51:12
588850    51303      4126  2010-12-31 23:57:07

[588851 rows x 3 columns]


# Full dataset

In [10]:


Data_Transpose = np.transpose(Data[:50000])
stream = ImplicitData(Data_Transpose[0],Data_Transpose[1])

num_clusters = {8,16,24}
cl_num_iterations = 10
cl_learn_rate = 0.1
cl_regularization = 0.1
num_factors = 160
num_iter = 4
learn_rate = 0.5
regularization = 0.4
overlap = {0.6,0.7,0.8}



In [11]:
%%capture out

tuning = {}

for num in num_clusters:
    tuning[num] = {}
    
    
    for val in overlap:

        print("")
        print("num_clusters = ", num, ", overlap = ", val)
        model = ICE_NMF(ImplicitData([],[]), num, val, cl_num_iterations, cl_learn_rate, cl_regularization, 
        num_factors, num_iter, learn_rate, regularization, random_seed = 100)
        eval = EvalPrequential(model,stream, metrics = ["Recall@20"])

        start_recommend = datetime.now()
        print('start time', start_recommend)

        results=eval.EvaluateTime(0,stream.size, 100)

        end_recommend = datetime.now()
        print('end time', end_recommend)

        tempo = end_recommend - start_recommend

        tuning[num][val] = np.mean(results['Recall@20'])
    
        print('npmean(resuls[Recall@20])', tuning[num][val])
        print('run time', tempo)
        #print('get tuple',np.mean(results['time_get_tuple']))
        print('recommend',np.mean(results['time_recommend']))
        #print('eval_point',np.mean(results['time_eval_point']))
        print('update',np.mean(results['time_update'])) 

In [12]:
print(out)


num_clusters =  8 , overlap =  0.6
start time 2023-11-23 13:01:57.783904
end time 2023-11-23 13:03:35.108638
npmean(resuls[Recall@20]) 0.22745901639344263
run time 0:01:37.324734
recommend 0.10179919004440308
update 0.0009449359273910522

num_clusters =  8 , overlap =  0.7
start time 2023-11-23 13:03:35.173760
end time 2023-11-23 13:05:32.281070
npmean(resuls[Recall@20]) 0.23366336633663368
run time 0:01:57.107310
recommend 0.12152031719094455
update 0.0011064303302764894

num_clusters =  8 , overlap =  0.8
start time 2023-11-23 13:05:32.332144
end time 2023-11-23 13:07:46.104822
npmean(resuls[Recall@20]) 0.1947261663286004
run time 0:02:13.772678
recommend 0.146398170482798
update 0.0012232240295410157

num_clusters =  16 , overlap =  0.6
start time 2023-11-23 13:07:46.157115
end time 2023-11-23 13:11:14.999350
npmean(resuls[Recall@20]) 0.21935483870967742
run time 0:03:28.842235
recommend 0.28314071983419437
update 0.0015340789985656739

num_clusters =  16 , overlap =  0.7
start tim

In [13]:
tuning

{8: {0.6: 0.22745901639344263,
  0.7: 0.23366336633663368,
  0.8: 0.1947261663286004},
 16: {0.6: 0.21935483870967742,
  0.7: 0.24795081967213115,
  0.8: 0.24748490945674045},
 24: {0.6: 0.1892430278884462,
  0.7: 0.24324324324324326,
  0.8: 0.2504930966469428}}